Установим `transformers`

In [1]:
!pip install transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.4/7.4 MB 55.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 26.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 107.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 81.0 MB/s eta 0:00:00


Далее установим необходимые модули и загрузим сам модуль

In [2]:
import torch
from transformers import AutoTokenizer, AutoModel, AutoModelWithLMHead, AutoModelForCausalLM, AutoModelForSeq2SeqLM, AutoModelForMaskedLM

model_name_or_path = "sberbank-ai/rugpt3large_based_on_gpt2"

tokenizer = AutoTokenizer.from_pretrained(model_name_or_path)

model = AutoModelForCausalLM.from_pretrained(model_name_or_path)
model.eval()
model.to('cuda')

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


GPT2LMHeadModel(
  (transformer): GPT2Model(
    (wte): Embedding(50257, 1536)
    (wpe): Embedding(2048, 1536)
    (drop): Dropout(p=0.1, inplace=False)
    (h): ModuleList(
      (0-23): 24 x GPT2Block(
        (ln_1): LayerNorm((1536,), eps=1e-05, elementwise_affine=True)
        (attn): GPT2Attention(
          (c_attn): Conv1D()
          (c_proj): Conv1D()
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )
        (ln_2): LayerNorm((1536,), eps=1e-05, elementwise_affine=True)
        (mlp): GPT2MLP(
          (c_fc): Conv1D()
          (c_proj): Conv1D()
          (act): NewGELUActivation()
          (dropout): Dropout(p=0.1, inplace=False)
        )
      )
    )
    (ln_f): LayerNorm((1536,), eps=1e-05, elementwise_affine=True)
  )
  (lm_head): Linear(in_features=1536, out_features=50257, bias=False)
)

Создадим обертку `generate` для удобства работы

In [14]:
def generate(text, max_length):
  # encode context the generation is conditioned on
  input_ids = tokenizer.encode(text, return_tensors='pt')
  input_ids = input_ids.to('cuda')

  # generate text until the output length (which includes the context length) reaches max_length
  greedy_output = model.generate(input_ids, max_length=max_length,no_repeat_ngram_size=5,
    repetition_penalty=2.)
  return tokenizer.decode(greedy_output[0], skip_special_tokens=True).split("<s>")[0] # очищаем от мусора в ответе

И можем проверить работу загруженного модуля

In [16]:
# @title Работа с моделью


start_text = "Кто такой Пушкин? " # @param {type:"string"}
gen_lenth = 800 # @param {type:"integer"}

text = generate(start_text,
         gen_lenth)
print(text)

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Кто такой Пушкин? 
- Это я, - сказал он.
